# GSP SEVT NETCONF/YANG

This notebook goes through a set of examples with a XRv9000 virtual platform. The goal is to show how NETCONF/YANG can be simply leveraged with IOS-XR to perform a range of tasks for the customer. We will cover topics like:

* Basic connectivity
* Why we really want to use some form of client library
* Getting started with a Pythin client like [ncclient](https://github/com/ncclient/ncclient)
* Model and feature discovery
* Querying and configuring features

First, let's talk a little about the environment being used here. This is all run inside **jupyter notebooks**, which some may already be familiar with as **iPython notebooks**.

A jupyter notebook lets us mix text and code and provides a simple way to experiment with Python code.

## Useful Snippets

Let's define some useful little snippets of code that we might want to run to define utilities or to enable debugs, set variables for use, etc.

### Enable Debugging

The ncclient libary can generate vast amounts of debug information via standard Python logging. The code fragment below shows how to enable this. But the logging is **vast**! You have been warned!

In [ ]:
import logging

handler = logging.StreamHandler()
for l in ['ncclient.transport.ssh', 'ncclient.transport.session', 'ncclient.operations.rpc']:
    logger = logging.getLogger(l)
    logger.addHandler(handler)
    logger.setLevel(logging.DEBUG)

### Pretty Printing XML

The ncclient library works with XML. It takes XML in and gives XML back. Yeah, I know it's not JSON, so we need to make it a little easier to read, so let's define a pretty printer we can use with ncclient responses:

In [ ]:
from lxml import etree

def pretty_print(nc_retval):
        print(etree.tostring(etree.fromstring(nc_retval.data_xml), pretty_print=True))

## Connecting to a Device

First, we need to make sure the device itself has NETCONF/YANG enabled. It's pretty simple:

```
ssh server v2
ssh server netconf
netconf-yang agent ssh
```

You also need to generate a host key from the exec prompt:

```
crypto key generate rsa
```

Next, let's define some variables that let us attach to the device. I'm connecting through an SSH tunnel to my device running on a UCS server in the lab. You should replace the variables below with the device of your choice:

In [ ]:
HOST = '127.0.0.1'
PORT = 8304
USER = 'cisco'
PASS = 'cisco'

In [ ]:
HOST = '172.27.255.38'
PORT = 830
USER = 'admin'
PASS = 'admin'

In [ ]:
# Giles' ASR9K
HOST = '172.23.29.104'
PORT = 830
USER = 'admin'
PASS = 'i2ssfail'

In [ ]:
HOST = '172.19.193.137'
PORT = 830
USER = 'admin'
PASS = 'cisco'

Now let's establish a NETCONF session to that box using ncclient:

In [ ]:
from ncclient import manager

def my_unknown_host_cb(host, fingerprint):
    return True
    
m = manager.connect(host=HOST, port=PORT, username=USER, password=PASS,
                    allow_agent=False,
                    look_for_keys=False,
                    hostkey_verify=False,
                    unknown_host_cb=my_unknown_host_cb)

## Capabilities

Let's look at the capabilities presented by the thing we've just connected to:

In [ ]:
print(m.server_capabilities)

Ok, that's a bit messy, so let's tidy it up a bit and look, initially, at all the base netconf capabilities:

In [ ]:
nc_caps = [c for c in m.server_capabilities if c.startswith('urn:ietf:params:netconf')]
for c in nc_caps:
    print(c)

And now let's look at the capabilities that are related to model support:

In [ ]:
import re

for c in m.server_capabilities:
    model = re.search('module=([^&]*)&', c)
    if model is not None:
        print("{}".format(model.group(1)))

        revision = re.search('revision=([0-9]{4}-[0-9]{2}-[0-9]{2})', c)
        if revision is not None:
            print("  revision = {}".format(revision.group(1)))

        deviations = re.search('deviations=([a-zA-Z0-9\-,]+)($|&)',c)
        if deviations is not None:
            print("  deviations = {}".format(deviations.group(1)))

        features = re.search('features=([a-zA-Z0-9\-,]+)($|&)',c)
        if features is not None:
            print("  features = {}".format(features.group(1)))

## Schema Discovery

Building on the parsing of the capabilities we saw above, NETCONF/YANG can also let a client discover more details on the schemas supported by a box.

But why do we care? Let's think back to what we talked about earlier. About the need for boxes to describe themselves to their clients. To expose their "model".

Let's pick a base model that looks like it may do something interesting, for example ```Cisco-IOS-XR-ifmgr-cfg```, and let's download the schema. The ncclient library provides a nice, simple function for that.

In [ ]:
# c = m.get_schema('Cisco-IOS-XR-pfi-im-cmd-oper')
c = m.get_schema('ned')

print(c.xml)

Yeah, that's all a bit messy, so let's tidy it up a bit, which means converting some HTML entities to the characters we're used to and extracting just the schema text from the XML ```<data>``` element:

In [ ]:
from BeautifulSoup import BeautifulStoneSoup

print BeautifulStoneSoup(c.xml, convertEntities=BeautifulStoneSoup.HTML_ENTITIES).find('data').getText()

## What About Config?

The ncclient library provides for some simple operations. Let's skip thinking about schemas and stuff like that. Instead let's focus on config and getting end setting it. For that, ncclient provides two methods:

* get_config - takes a target data store and an optional filter
* edit_config - takes a target data store and an XML document with the edit request


### Getting Config

Let's look at some simple requests...

In [ ]:
c = m.get_config(source='running')
pretty_print(c)

Now let's add in a simple filter to just get interface configuration:

In [ ]:
filter = '''
<interface-configurations xmlns="http://cisco.com/ns/yang/Cisco-IOS-XR-ifmgr-cfg"/>
'''
c = m.get_config(source='running', filter=('subtree', filter))
pretty_print(c)

Let's refine the query a bit to look at **just** the management ethernet port:

In [ ]:
filter = '''
<interface-configurations xmlns="http://cisco.com/ns/yang/Cisco-IOS-XR-ifmgr-cfg">
  <interface-configuration>
    <interface-name>MgmtEth0/RP0/CPU0/0</interface-name>
  </interface-configuration>
</interface-configurations>
'''
c = m.get_config(source='running', filter=('subtree', filter))
pretty_print(c)

Or we may want to look at just ISIS config:

In [ ]:
filter = '''<isis xmlns="http://cisco.com/ns/yang/Cisco-IOS-XR-clns-isis-cfg"/>'''
c = m.get_config(source='running', filter=('subtree', filter))
pretty_print(c)

Or maybe just L2VPN?

In [ ]:
filter = '''<evpn xmlns="http://cisco.com/ns/yang/Cisco-IOS-XR-l2vpn-cfg"/>'''
c = m.get_config(source='running', filter=('subtree', filter))
pretty_print(c)

### Editing Config

Let's start with something simple, like applying an IP address to an interface and bringing it up. We can actually use the data we got from the management ethernet port as a template.

To do this, we use two functions:

* ```edit_config``` on the candidate data store
* ```commit``` to commit the change to running config, just like on the CLI

For good measure, we'll also get the interface config back again to check out that what we asked to happen, actually did happen.

In [ ]:
from ncclient.operations import TimeoutExpiredError

edit_data = '''
<config>
  <interface-configurations xmlns="http://cisco.com/ns/yang/Cisco-IOS-XR-ifmgr-cfg">
   <interface-configuration>
    <active>act</active>
    <interface-name>GigabitEthernet0/0/0/0</interface-name>
    <ipv4-network xmlns="http://cisco.com/ns/yang/Cisco-IOS-XR-ipv4-io-cfg">
     <addresses>
      <primary>
       <address>192.168.1.10</address>
       <netmask>255.255.255.0</netmask>
      </primary>
     </addresses>
    </ipv4-network>
    <shutdown nc:operation="remove"/>
   </interface-configuration>
  </interface-configurations>
</config>
'''

filter = '''
<interface-configurations xmlns="http://cisco.com/ns/yang/Cisco-IOS-XR-ifmgr-cfg">
  <interface-configuration>
    <interface-name>GigabitEthernet0/0/0/0</interface-name>
  </interface-configuration>
</interface-configurations>
'''

try:
    m.edit_config(edit_data, target='candidate', format='xml')
    m.commit()
    c = m.get_config(source='running', filter=('subtree', filter))
    pretty_print(c)
except TimeoutExpiredError as e:
    print("Operation timeout!")
except Exception as e:
    print("severity={}, tag={}".format(e.severity, e.tag))
    print(e)

And now what about deleting that config and taking the interface back to it's original state?

In [ ]:
from ncclient.operations import TimeoutExpiredError

filter = '''
<interface-configurations xmlns="http://cisco.com/ns/yang/Cisco-IOS-XR-ifmgr-cfg">
  <interface-configuration>
    <interface-name>GigabitEthernet0/0/0/0</interface-name>
  </interface-configuration>
</interface-configurations>
'''

edit_data = '''
<config>
  <interface-configurations xmlns="http://cisco.com/ns/yang/Cisco-IOS-XR-ifmgr-cfg">
   <interface-configuration>
    <active>act</active>
    <interface-name>GigabitEthernet0/0/0/0</interface-name>
    <ipv4-network xmlns="http://cisco.com/ns/yang/Cisco-IOS-XR-ipv4-io-cfg" nc:operation="remove"/>
    <shutdown/>
   </interface-configuration>
  </interface-configurations>
</config>
'''


edit_data2 = '''
<config>
  <interface-configurations xmlns="http://cisco.com/ns/yang/Cisco-IOS-XR-ifmgr-cfg">
   <interface-configuration>
    <active>act</active>
    <interface-name>GigabitEthernet0/0/0/0</interface-name>
    <shutdown nc:operation="delete"/>
    <ipv4-network xmlns="http://cisco.com/ns/yang/Cisco-IOS-XR-ipv4-io-cfg" nc:operation="delete"/>
   </interface-configuration>
  </interface-configurations>
</config>
'''


try:
    m.edit_config(edit_data2, target='candidate', format='xml')
    m.commit()
    c = m.get_config(source='running', filter=('subtree', filter))
    pretty_print(c)
except TimeoutExpiredError as e:
    print("Operation timeout!")
except Exception as e:
    print("severity={}, tag={}".format(e.severity, e.tag))
    print(e)

## What About Oper Data?

As we touched on before, NETCONF also has the ```get``` operation. This can get both configuration state **and** operational state.

### Verbose Interface Stats

In [ ]:
from ncclient.operations import TimeoutExpiredError

filter = '''
<interfaces xmlns="http://cisco.com/ns/yang/Cisco-IOS-XR-pfi-im-cmd-oper"/>
'''

try:
    c = m.get(filter=('subtree', filter))
    pretty_print(c)
except TimeoutExpiredError as e:
    print("Operation timeout!")
except Exception as e:
    print("severity={}, tag={}".format(e.severity, e.tag))
    print(e)

### Slightly Less Verbose Interface Stats

In [ ]:
from ncclient.operations import TimeoutExpiredError

filter = '''
<interfaces xmlns="http://cisco.com/ns/yang/Cisco-IOS-XR-pfi-im-cmd-oper">
  <interface-briefs/>
</interfaces>
'''

try:
    c = m.get(filter=('subtree', filter))
    pretty_print(c)
except TimeoutExpiredError as e:
    print("Operation timeout!")
except Exception as e:
    print("severity={}, tag={}".format(e.severity, e.tag))
    print(e)

### Zeroing In On A Specific Interface

In [ ]:
from ncclient.operations import TimeoutExpiredError

filter = '''
<interfaces xmlns="http://cisco.com/ns/yang/Cisco-IOS-XR-pfi-im-cmd-oper">
  <interface-xr>
    <interface>
      <interface-name>MgmtEth0/RP0/CPU0/0</interface-name>
      <interface-statistics/>
    </interface>
  </interface-xr>
</interfaces>
'''

try:
    c = m.get(filter=('subtree', filter))
    pretty_print(c)
except TimeoutExpiredError as e:
    print("Operation timeout!")
except Exception as e:
    print("severity={}, tag={}".format(e.severity, e.tag))
    print(e)

### Zeroing In On Specific Stats Across Interfaces

In [ ]:
from ncclient.operations import TimeoutExpiredError

filter = '''
<interfaces xmlns="http://cisco.com/ns/yang/Cisco-IOS-XR-pfi-im-cmd-oper">
  <interface-xr>
    <interface>
      <interface-name>MgmtEth0/RP0/CPU0/0</interface-name>
      <interface-statistics>
        <full-interface-stats>
          <packets-received/>
          <bytes-received/>
          <packets-sent/>
          <bytes-sent/>
        </full-interface-stats>
      </interface-statistics>
    </interface>
  </interface-xr>
</interfaces>
'''

try:
    c = m.get(filter=('subtree', filter))
    pretty_print(c)
except TimeoutExpiredError as e:
    print("Operation timeout!")
except Exception as e:
    print("severity={}, tag={}".format(e.severity, e.tag))
    print(e)

## Conclusion

NETCONF presents a number of useful primitives that we looked at today:

* get config
* get
* edit config
* commit
* get schema

There are more, so follow up to understand more later.

With the prmitives we ran through today, you can do basic model discovery, understand what features are supported, and understand what parts of models are not supported.

Products such as NSO, or open source projects like ODL or the YDK, can use these basic capabilities to work with devices in a much more reliable way.